<a href="https://colab.research.google.com/github/UsaMasa/Python/blob/master/KalmanFilter%E5%85%A5%E9%96%80%E3%83%8E%E3%83%BC%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kalman Filter 入門
カルマンフィルタとは、誤差を持った観測値を用いて、ある動的なシステムの状態をより正確に推定するための効率的な再帰アルゴリズムの一種である。

[1]によると、「ベイズの定理の応用によって数理モデルと観測値から精度良くシステムの状態推定を行うプログラム」と解釈できる。さらに誤解を恐れず平たく言うなら、観測量と予測量を再帰的に計算した最適な重みを用いて重み付け平均していくようなアルゴリズムである。今回は[1]の記事に基づいて勉強する。

### ベイズの定理
P(B|A)を事象Aが起きた時に事象Bが起きる条件付き確率を表すとすると、ベイズの定理は以下の通りとなる
$$
  P(B|A) = \frac{P(A|B)}{P(A)} P(B)
$$
この時、左辺を事後確率、右辺のP(B)をP(B|A)の事前確率と呼ぶ。
この辺りの例示は[2]などがわかりやすい。

ベイズの定理は従来の分布を仮定してそれに基づいた主張を行う頻度主義に対し、サンプリングに応じて分布の推定を修正していく点に特徴がある。

## 数理モデル
カルマンフィルタを用いるためには対象の数理モデルを理解する必要がある。数理モデルとは、例えば状態変異や観測の数学的表現、ノイズの性質等があげられる。
特に、線形のカルマンフィルタを用いる際には以下の4つの制約がかけられる。

1.   システムは線形の動的システムであること
2.   ホワイトノイズであること
3.   初期状態の事前分布がガウス分布であること
4.   フィルタ後の分布もガウス分布に従うこと

これは素粒子実験とも相性が良い制約である。素粒子の振る舞いに関しては（線形近似して）定式化が可能である。実験における大半の観測ノイズはホワイトノイズであり、ほとんどの測定状態は（検出器の較正さえきちんとしていれば）ガウス分布に従うはずである。


## 定式化
$$
\mathbf{
  x_{t+1} = A_tx_t + Bu_t + w_t \\
  y_t = Cx_t + v_t \\
  w_t = \mathit{N}(0,Q)\\
  v_t = \mathit{N}(0,R)\\
  \mathit{p}(x_t) = \mathit{N}(x_t | m_0, V_0)
}
$$
というシステムを考える。この時、xに関する式を状態の遷移に関する方程式（状態方程式、odometry）と呼び、yに関する式を観測による方程式（観測方程式、observation）という。すなわち、状態方程式によって事前分布を更新し、観測方程式によって観測の効果を入れた事後分布を更新するのである。

簡単な例として、[3]のトロッコの問題では物体の運動方程式を状態方程式としているので、
$$
  {\mathbf x_t} =  \left(
    \begin{array}{c}
      x_t  \\
      v_t  \\
    \end{array}
  \right)
$$

$$
  \begin{align}
  \mathbf{x_{t+1}} &= 
  \left[
    \begin{array}{cc}
      1 & dt  \\
      0 & 1  
    \end{array}
  \right] \mathbf{x_t}+  
  \left[
  \begin{array}{c}
      \frac{1}{2}dt^2  \\
      dt
    \end{array}
  \right] a_t +\mathbf{w_t} \\
  &= \mathbf{A_tx_t^{true} + Bu_t + w_t}
  \end{align}
$$
といったように状態方程式が立てられる。観測方程式としてはセンサの観測情報にノイズを乗せたものを仮定しているので、
$$
  \mathbf{y_t = H_t x_t^{true} + v_t}
$$
と与えられる。

## カルマンフィルタの導出

### 定理の整理
カルマンフィルタの導出には以下の二つの定理が必要になる。（どちらも証明はPRML[4]に書かれているので省略する。）



1.  多変量ガウス分布の線形結合に関する定理
2.  多変量ガウス分布の条件付き分布に関する定理
3.  周辺分布・条件付き分布に対する同時分布の定理


**定理1.**

$$ 
\mathbf{
  y = Ax + b + v} \\
  p(\mathbf{x}) = N(\mathbf{x} | \mathbf{m, P})\\
  p(\mathbf{v}) = N(\mathbf{v} | \mathbf{0, R})
$$
の時、yの事前確率は
$$
  p(\mathbf{y}) = N(\mathbf{y} | \mathbf{Am+b, APA^T + R})
$$


**定理2.**

ベクトル$\mathbf{x,y}$の同時確率分布が

$$
  p(\mathbf{x,y}) = N( 
  \left[\begin{array}{c}
  \mathbf{x \\
  y}
  \end{array}
  \right]
  |
  \left[\begin{array}{c}
  \mathbf{a \\
  b}
  \end{array}
  \right]
  ,
  \left[
    \begin{array}{cc}
      \mathbf{P} & \mathbf{R}  \\
      \mathbf{R^T} & \mathbf{Q}
    \end{array}
  \right]
  )
$$
に従うとする。
この時、$\mathbf{x,y}$の周辺分布は、

$$
  p(\mathbf{x}) = N(\mathbf{x} | \mathbf{a, P})\\
  p(\mathbf{y}) = N(\mathbf{y} | \mathbf{b, Q})
$$
と書くことができる。ここで、一方の値が得られた時、もう一方の事後確率分布を計算することができ、例えばyを得られたならば、xは

$$
  p(\mathbf{x | y}) = N(\mathbf{x} | \mathbf{a+RQ^{-1}(y-b), P-RQ^{-1}R^T})
$$
逆にxを得たならば、
$$
  p(\mathbf{y | x}) = N(\mathbf{y} | \mathbf{b+R^TP^{-1}(x-a), Q-R^TP^{-1}R})
$$
となる、これにより、状態量の事前分布がわかれば観測が得られた時に、事後分布を求めることが可能になる。ちなみにこれらの式は、一方の観測情報が得られた時に、一方の分散がその情報分（分散分）小さくなることを示している。

**定理3**

　[4]の(2.104)より。
$$
p(x) = N(x | m,V^{-1}) \\
p(y | x) = N(y | Ax+b, L^{-1})
$$
とした時の
$$
z = [x, y]^T
$$
の同時分布の共分散行列は、
$$
R = \left[
    \begin{array}{cc}
      \mathbf{V + A^TLA} & \mathbf{-A^TL}  \\
      \mathbf{-LA} & \mathbf{L}
    \end{array}
  \right]
$$
の逆行列
$$
cov[\mathbf{z}] = \left[
    \begin{array}{cc}
      \mathbf{V^{-1}} & \mathbf{V^{-1}A^T}  \\
      \mathbf{AV^{-1}} & \mathbf{L^{-1} + AV^{-1}A^T}
    \end{array}
  \right]
$$
になる。定理3は定理1の帰結であるが、この形がそのまま後半用いられるのでここに示しておく。

### カルマンフィルタの導出

以下の2ステップに基づいて、事後分布の推定を行う。


1.   状態方程式による状態推移の推定（事前分布の計算）
2.   観測方程式による、観測結果に基づいた推定の更新（観測更新）



**目標**

時刻tにおける入力を受け取った時に、時刻t＋1の状態を推定する。すなわち、時刻tまでの観測で、
$$
  p(x_t | y_t) = N(x_t | m_t, V_t) 
$$
がわかっている時に、時刻t＋1の観測に基づいて推定を更新し、
$$
  p(x_{t+1} | y_{t+1}) = N(x_{t+1} | m_{t+1}, V_{t+1})
$$
を得られることを示す。

**状態遷移の予測**

時刻tでの状態方程式は以下の通り。
$$
\mathbf{
  x_{t+1} = Ax_t + Bu_t + w_t\\
  w_t \sim N(0,Q)
}
$$
ここで定理1より、
$$
\begin{align}
  p(\mathbf{x_{t+1} | y_t}) &= N(\mathbf{x_t+1 | Am_t+Bu_t, AV_tA^T + Q})\\
  &=N(\mathbf{x_{t+1} | m^{est}_{t+1}, V^{est}_{t+1}}) \\
  \mathbf{m^{est}_{t+1}} &= \mathbf{Am_t+Bu_t} \\
  \mathbf{V^{est}_{t+1}} &= \mathbf{AV_tA^T + Q}
  \end{align}
$$
これによって、$\mathbf{x_{t+1}}$の事前分布が得られた。
少しconfusingなのが、条件付き確率で得られている点だが、$y_{t+1}$という観測に対する、という観点でその事後分布の計算に再帰的に用いられる事前分布である。

**観測更新**

次に上で求めた状態量の予測分布から、観測量と状態量$\mathbf{y_{t+1},x_{t+1}}$の同時予測分布$p(\mathbf{x_{t+1},y_{t+1} | y_t})$を求める。その後、観測値$y_{t+1}$を用いて目的とする分布$p(\mathbf{x_{t+1} | y_{t+1}})$を求める。前者には定理3を、後者には定理2を用いる。順に見ていく。

まず、観測方程式は
$$
\mathbf{
  y_{t+1} = Cx_{t+1} + v_{t+1}\\
  v_{t+1} \sim \mathit{N}(0,R)
}
$$
この時、$\mathbf{y_{t+1},x_{t+1}}$の同時予測分布は、定理3.より、
$$
  p(\mathbf{y_{t+1},x_{t+1} | y_t}) = N(
    \left[
    \begin{array}{c}
      \mathbf{x_{t+1}}  \\
      \mathbf{y_{t+1}} 
    \end{array}
    \right]
     |
     \left[ 
     \begin{array}{c}
      \mathbf{x_{t+1}}  \\
      \mathbf{y_{t+1}} 
    \end{array}
    \right]
    \left[
     \begin{array}{cc}
      \mathbf{V_{t+1}^{est}} & \mathbf{V_{t+1}^{est} C^T}  \\
      \mathbf{C V_{t+1}^{est}} & \mathbf{CV_{t+1}^{est}C^T + R}
    \end{array}
    \right]
  )
$$


こうして同時予測分布の形が得られたので、次に定理2を用いて
$$
  p(\mathbf{x_{t+1} | y_{t+1}}) = N(\mathbf{x_{t+1} | m_{t+1}^{est} + K_{t+1} (y_{t+1} - Cm_{t+1}^{est}), (I - K_{t+1} C) V_{t+1}^{est}}) \\
  \mathbf{K_{t+1}} = \mathbf{V_{t+1}^{est}C^T (CV_{t+1}^{est}C^T + R)^{-1}}
$$
と書くことが出来た。ここで、$\mathbf{K_{t+1}}$をカルマンゲインと呼び、数学的に最適な観測値と予測値の間の重みである。


### Reference
[1] http://koukyo1213.hatenablog.com/entry/KalmanFilter \\
[2] https://bellcurve.jp/statistics/course/6446.html \\
[3] https://qiita.com/IshitaTakeshi/items/740ac7e9b549eee4cc04 \\
[4] パターン認識と機械学習　上　C.M.ビショップ著（丸善出版）

# 実装


## 問題設定
WIP